In [1]:
import jsonlines
from dialop.games import OptimizationGame
from dialop.games.optimization import WORKERS, TASKS

with jsonlines.open('data/optimization.jsonl') as jsonl_f:
     data = [obj for obj in jsonl_f]
data = data[:70]

In [2]:
import random
idx = random.randint(0, len(data))

def stringify_proposal(assignment):
    assignment.sort(key=lambda x: x[1])
    out_str = ""
    out_str = '\n'.join([f"{TASKS[a[1]]}: {WORKERS[a[0]]} ({game.table.values[a[0]][a[1]]})" for a in assignment])
    return out_str

d = data[idx]
print(f"Dialog idx: {idx}")

game = OptimizationGame.create_from_game_state(d)

table = game.tables[0]
obs_string = "Player 1 knowledge: \n" 
for row in table[1:]:
    obs_string += row[0] + ": "
    obs_string += ', '.join([f"{c} for {TASKS[i]}" for i, c in enumerate(row[1:]) if type(c) == int])
    obs_string += " \n "
print(obs_string)

table = game.tables[1]
obs_string = "Player 2 knowledge: \n" 
for row in table[1:]:
    obs_string += row[0] + ": "
    obs_string += ', '.join([f"{TASKS[i]}:{c}" for i, c in enumerate(row[1:]) if type(c) == int])
    obs_string += " \n "
print(obs_string)

print(f"{'-'*50} CONVERSATION {'-'*50}")
final_proposal = None
for a in d['action_log']:
    output_str = f"Player {a['player']+1}: "
    if a['type'] == 'message':
        output_str += a['message']['data']
    elif a['type'] == 'proposal':
        output_str += a['proposal'].replace('<br/>&emsp;', '\n')
        final_proposal = a['proposal_ids']
    elif a['type'] == 'proposal_response':
        output_str += 'ACCEPT' if a['response']['accept'] else 'REJECT'
    print(output_str)

print("-"*100)
best_assignment, best_reward = game.table.find_max_value_known_assignment([d['mask1'], d['mask2']])
print("\nFINAL PROPOSAL:")
print(stringify_proposal(final_proposal))
print(f"Proposal reward = {d['proposal_reward']}")
print("\nOPTIMAL ASSIGNMENT:")
print(stringify_proposal(best_assignment))
print(f"Best possible reward = {d['best_assignment_reward']}")

Dialog idx: 14
Player 1 knowledge: 
Ava Li: 125 for Electra 
 Daniel Nguyen: 237 for LLaMA, 232 for SWAG 
 Sofia Patel: 199 for Electra, 125 for SWAG 
 Andrei Petrov: 43 for GLUE 
 Morgan Reed: 120 for BLEU, 33 for GloVe, 84 for SWAG 
 Joseph Santos: 224 for GloVe, 68 for GLUE, 199 for LLaMA, 17 for RoBERTa, 209 for SWAG 
 Ethan Smith: 84 for GloVe, 212 for QuAC, 40 for SWAG 
 Noah Wilson: 74 for Electra, 12 for GLUE, 40 for RoBERTa 
 
Player 2 knowledge: 
Ava Li: BLEU:159, GLUE:304, RoBERTa:206 
 Daniel Nguyen: Electra:186, GLUE:23, LLaMA:314, RoBERTa:145, QuAC:142 
 Sofia Patel: Electra:263, GloVe:43, LLaMA:6, QuAC:135 
 Andrei Petrov: BLEU:290, QuAC:311 
 Morgan Reed: GLUE:30, SWAG:111 
 Joseph Santos: BLEU:203, Electra:196, QuAC:94 
 Ethan Smith: GloVe:111, RoBERTa:159 
 Noah Wilson: Electra:98, GLUE:16, SWAG:6 
 
-------------------------------------------------- CONVERSATION --------------------------------------------------
Player 2: Hey!
Player 1: Hi
Player 2: Can you help me w

In [3]:
import numpy as np
scores = [d['result']['norm'] for d in data]
print(np.average(scores), np.std(scores))

0.9239836102326444 0.09391325964207785


In [4]:
turns = [len([a  for a in d['action_log'] if a['type'] == 'message']) for d in data]
print(np.average(turns), np.std(turns))

13.114285714285714 7.974141883332036


In [5]:
words = [sum([len(a['message']['data'].split(' ')) for a in d['action_log'] if a['type'] == 'message']) for d in data]
print(np.average(words), np.std(words))
print(words)

184.3 131.65543231155658
[492, 225, 107, 269, 146, 223, 158, 64, 267, 48, 180, 619, 224, 199, 82, 174, 167, 127, 88, 484, 284, 376, 31, 49, 187, 53, 239, 402, 56, 105, 206, 254, 104, 174, 41, 257, 56, 153, 179, 137, 46, 121, 558, 86, 143, 268, 555, 154, 140, 64, 140, 89, 98, 53, 183, 151, 60, 48, 117, 217, 95, 83, 114, 326, 191, 129, 135, 376, 147, 328]
